In [5]:
!pip install -q requests torch bitsandbytes transformers accelerate sentencepiece

#### The below import is not working, need to get this fixed in MAC

In [ ]:
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
import os
from dotenv import load_dotenv

ImportError: dlopen(/opt/anaconda3/envs/llms/lib/python3.11/site-packages/torch/_C.cpython-311-darwin.so, 0x0002): Library not loaded: @rpath/libgfortran.5.dylib
  Referenced from: <CAA510D4-816A-34E5-9021-485EF5D56159> /opt/anaconda3/envs/llms/lib/libopenblas.0.dylib
  Reason: tried: '/opt/anaconda3/envs/llms/lib/libgfortran.5.dylib' (duplicate LC_RPATH '@loader_path'), '/opt/anaconda3/envs/llms/lib/libgfortran.5.dylib' (duplicate LC_RPATH '@loader_path'), '/opt/anaconda3/envs/llms/lib/libgfortran.5.dylib' (duplicate LC_RPATH '@loader_path'), '/opt/anaconda3/envs/llms/lib/libgfortran.5.dylib' (duplicate LC_RPATH '@loader_path'), '/opt/anaconda3/envs/llms/lib/libgfortran.5.dylib' (duplicate LC_RPATH '@loader_path'), '/opt/anaconda3/envs/llms/lib/libgfortran.5.dylib' (duplicate LC_RPATH '@loader_path'), '/opt/anaconda3/envs/llms/lib/libgfortran.5.dylib' (duplicate LC_RPATH '@loader_path'), '/opt/anaconda3/envs/llms/lib/libgfortran.5.dylib' (duplicate LC_RPATH '@loader_path'), '/opt/anac

In [16]:
# retrieve the token from the environment variable
from dotenv import load_dotenv
load_dotenv()
# Load the environment variables from .env file
hf_token = os.getenv('HF_TOKEN')
if hf_token is None:
    raise ValueError("HF_TOKEN environment variable not set. Please set it to your Hugging Face token.")
login(token=hf_token, add_to_git_credential=True)

NameError: name 'os' is not defined

In [ ]:
# import MIXTRAIL AND QWEN2 model name
GEMMA_2 = "google/gemma-2-it"
LLMAMA = "llama/llama-2-7b-Instruct"
QWEN_2 = "Qwen/Qwen2-7B-Instruct"
MIXTRAIL = "mistral/Mistral-8x7B-Instruct-v0.1"

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Tell a light-hearted joke for a room of data Scientists."},
]

In [ ]:
# Accessing LLama 3.1 from Meta

# Quantization Config = this allows us to load the model into memory and use less memory

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
# reduce the 32bit model to 4 bit, this will reduce the memory usage by 8x and will reduce
# the accuracy but it is still tolerable, this is a trade off between memory and accuracy
# Load the model


In [ ]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    LLMAMA
)
tokenizer.pad_token = tokenizer.eos_token
# Load the model
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt")

In [ ]:
# The model
# LLM which takes the input and generates the output - a generic kind of LLM - AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    LLMAMA,
    quantization_config=quant_config,
    device_map="auto"
)

In [ ]:
memory = model.get_memory_footprint() / 1e6
print(f"Memory footprint: {memory:.2f} MB")

In [ ]:
model
# the above code gives the description of the model

In [ ]:
outputs = model.generate(inputs, max_new_tokens = 80)
# The model generates the output
print(tokenizer.decode(outputs[0]))

In [ ]:
# clean up the memory
del inputs, outputs, model
torch.cuda.empty_cache()

In [ ]:
# wrapping everything in a function - and adding Streaming

def generate(model, messages):
    tokenizer = AutoTokenizer.from_pretrained(
        LLMAMA
    )
    tokenizer.pad_token = tokenizer.eos_token
    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt")
    # The model
    # LLM which takes the input and generates the output - a generic kind of LLM - AutoModelForCausalLM
    streamer = TextStreamer(tokenizer)
    model = AutoModelForCausalLM.from_pretrained(
        LLMAMA,
        quantization_config=quant_config,
        device_map="auto",
        quantization_config=quant_config,
    )
    outputs = model.generate(inputs, max_new_tokens=80, streamer=streamer)
    del tokenizer, inputs, outputs, model
    torch.cuda.empty_cache()
    return outputs
# The model generates the output

In [ ]:
generate(model, messages)
# The model generates the output

In [ ]:
messages = [
    {"role": "user", "content": "Tell a light-hearted joke for a room of data Scientists."},
]
generate(GEMMA_2, messages)